In [75]:
#Modules used
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import urllib.request
from selenium.webdriver.common.by import By
import requests
import numpy as np
import time as t
import os.path
import comtypes.client
import requests
from bs4 import BeautifulSoup

In [67]:
#set download directory path
p = r"L:\PRIV\ToxValDB\Document Webscrape\source_opp2"
#Saves webpage urls and chemical names as key value pairs in links dictionary
links = {}

#Selenium chrome driver settings
s=Service(r'C:\Users\mmille16\Documents\chromedriver.exe')
Options = webdriver.ChromeOptions()
Options.headless = True
driver = webdriver.Chrome(service=s, options = Options)

#Open the linked website and access the data in the table
URL = 'https://www.epa.gov/sdwa/2021-human-health-benchmarks-pesticides'
driver.get(URL)
a = driver.find_elements(By.XPATH,"//table/tbody[@id = 'data']/tr/td/a")

#Save the final pdf links for metadata upload
doc_url = {}


for ele in a:
#Linked urls in the table could link to the pdfs or a different website to download the file
    landing = ele.get_attribute('href')
#Linked url is the pdf document url. Easy case
    if str(landing).lower().find('.pdf') != -1:
        chem_name = ele.text
        name = chem_name
        pdf_link = landing
#Url is not the pdf document url. Might just need to do some string manipulation
    else:
        name = ele.text
#Strip out the "Exit EPA's website" string hidden in the text and remove any slashes
        bad = "Exit EPA's website"
        name = name[0:-len(bad)]
        chem_name = name.replace('/','')
#PDF url could be found with the unique identifier at the end of the linked webpage url (EPA-HQ-numbers)
        landing = ele.get_attribute('href').split('/')[-1]
        pdf_name = landing[landing.find('EPA'):]
        pdf_link = f'https://downloads.regulations.gov/{pdf_name}/content.pdf'
    #links[chem_name] = ele.get_attribute('href')
    if os.path.isfile(f'{p}/{chem_name}_source_opp.pdf'): #Don't redo work we have already done
        continue
    try:
#Try to save the pdf url as a pdf in the file folder
        response = urllib.request.urlopen(pdf_link)
        file = open(f'{p}/{chem_name}_source_opp.pdf', 'wb')
        file.write(response.read())
        file.close()
        doc_url[name] = pdf_link
    except:
#Problematic chemicals. Will need to use Selenium and inspecting to see what is up with these.
        links[chem_name] = ele.get_attribute('href')
        continue
        #print(f'no short cut way to download origin document for {chem_name}')
    t.sleep(0.5)
driver.close()
print('done')

done


In [68]:
#Straggler catching. Not headless so I can see what may be "wrong" with some of these
#Save path
p = r"L:\PRIV\ToxValDB\Document Webscrape\source_opp2"

#Selenium chrome driver settings
s=Service(r'C:\Users\mmille16\Documents\chromedriver.exe')
Options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=s, options = Options)

#Will hold url and chem name for final missing chemcicall opp origin documents
missing = {}

for k,a in links.items():
#Open the first landing page for the chemical linked in the original table
    driver.get(a)
#This website takes forever to load so long sleep time is required
    t.sleep(20)
    try:
#Look for the download button here
        btn = driver.find_element(By.LINK_TEXT,'Download Icon Download')
        #print(f'regular regulations landing page contains pdf for {k}')
    except:
        try:
#or look for the download button here
            btn = driver.find_element(By.LINK_TEXT,'PDF')
            #print(f'different landing page contains pdf for {k}')
        except:
#Simply cannot find the a download link
            print(f'could not find a document link for {k}')
            missing[k] = a
            continue
#Prevent doing work I've already done
    if os.path.isfile(f'{p}/{k}_source_opp{btn.get_attribute("href")[-4:]}'):
        continue
#Download the file with a requests.get. 
    response = requests.get(btn.get_attribute('href'))
#Some of these files cannot be downloaded as pdfs first so use the original extension (.doc or .pdf)
    file = open(f'{p}/{k}_source_opp{btn.get_attribute("href")[-4:]}', 'wb')
    file.write(response.content)
    file.close()
    doc_url[k] = btn.get_attribute("href")
    #print(f'source document downloaded for {k}')
driver.close()
print('done')

could not find a document link for Furilazole
done


In [69]:
#Some origin docs had to be downloaded as .doc files instead of .pdf

directory1 = r"L:\PRIV\ToxValDB\Document Webscrape\source_opp2"
wdFormatPDF = 17

#Look for .doc files in the directory
for file in os.listdir(directory1):
    if str(file).lower().find('.doc') != -1:
#Current file path/name and future file path/name
        in_file = os.path.join(directory1,file)
        out_file = os.path.join(directory1,file.replace('.doc','.pdf'))
#Save each word doc as a pdf instead using comtypes.client
        word = comtypes.client.CreateObject('Word.Application')
        word.Visible = False
        t.sleep(3)
        doc=word.Documents.Open(in_file)
        doc.SaveAs(out_file, FileFormat=wdFormatPDF)
        doc.Close()
#Remove the old .doc file. No longer necessary since only pdfs are requested.
        os.remove(in_file)

In [84]:
#Getting metadata for chemicals that have been downloaded
url = "https://www.epa.gov/sdwa/2021-human-health-benchmarks-pesticides"
#Parse html with beautiful soup
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
#Go through and remove any attrbutes that are super scripts
for sup in soup.select('sup'):
    sup.extract()
#Final cleaned up table
table = pd.read_html(str(soup))[0]

#Creating the "info" dataframe containing chemicals and urls for documents that have been successfully downloaded
d = {'Chemical':doc_url.keys(),
    'URL':doc_url.values()}
info = pd.DataFrame(data = d)

#Metadata merges the cas numbers from the webpage table with the chemicals that have successfully downloaded origin docs
metadata = info.merge(table[['CAS Number',table.columns[0]]],
                     left_on = 'Chemical',
                     right_on = table.columns[0],
                     how = 'left').drop(table.columns[0],axis = 1)
metadata['Source'] = ['OPP']*len(metadata)
#This column will be necessary for uploading the metadata. Will need to merge with clowder log based on file name
metadata['file_name'] = [f'{str(x).replace("/","")}_source_opp.pdf' for x in metadata['Chemical']]

In [86]:
#Save the metadata datatframe as an excel spreadsheet
out_path = r"C:\Users\mmille16\OneDrive - Environmental Protection Agency (EPA)\Profile\Desktop\source_opp_metadata.xlsx"
with pd.ExcelWriter(out_path) as writer:
    metadata.to_excel(writer, index = False)